In [1]:
import numpy as np
import pandas as pd
import scipy as sci
import matplotlib.pyplot as plt
import os
import altair as alt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn import set_config
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split


# alt.data_transformers.enable('vegafusion')
# alt.renderers.enable('default')
# set_config(transform_output="pandas")

print("packages imported")

packages imported


In [3]:
#reading players.csv data
url_players='https://drive.google.com/file/d/1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz/edit'
url_players='https://drive.google.com/uc?id=' + url_players.split('/')[-2]
players = pd.read_csv(url_players)

#reading sessions.csv data
url_sessions='https://drive.google.com/file/d/14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB/edit'
url_sessions='https://drive.google.com/uc?id=' + url_sessions.split('/')[-2]
sessions = pd.read_csv(url_sessions)

#mapping experiences
experience_mapping = {
    'Beginner': 1,
    'Amateur': 2,
    'Regular': 3,
    'Veteran': 4,
    'Pro': 5
}

#converting dates to datetime object
sessions['start_time'] = pd.to_datetime(sessions['start_time'], format="%d/%m/%Y %H:%M")
sessions['end_time'] = pd.to_datetime(sessions['end_time'], format="%d/%m/%Y %H:%M")

sessions['session_length'] = (sessions['end_time'] - sessions['start_time']).dt.total_seconds() / 3600
sessions['original_session_length'] = (sessions['original_end_time'] - sessions['original_start_time'])

#grouping datas in sessions dataframe based onemail and finding summaries
player_sessions = sessions.groupby('hashedEmail').agg(
    number_sessions=('session_length', 'size'), 
    mean_session_length=('session_length', 'mean'), 
    sd_session_length=('session_length', 'std') 
).reset_index()

#merging the datas we need and tridying them
players_combined = pd.merge(players, player_sessions, on='hashedEmail', how='left')
players_combined['experience_val'] = players_combined['experience'].map(experience_mapping)
players_combined['subscribe_binary'] = players_combined['subscribe'].astype(int)
players_combined = players_combined.dropna(subset=['subscribe_binary', 'experience_val', 'age', 'number_sessions', 'played_hours'])
players_combined.head(5)

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName,number_sessions,mean_session_length,sd_session_length,experience_val,subscribe_binary
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN,27.0,1.246296,0.902162,5,1
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN,3.0,1.416667,1.233671,4,1
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN,1.0,0.083333,NaN,4,0
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN,1.0,0.833333,NaN,2,1
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN,1.0,0.150000,NaN,3,1


In [4]:
#splitting the testing and training data with ratio 7:3
players_training, players_testing = train_test_split(
   players_combined, test_size=0.3, random_state=2000 
)

X_train = players_training[["age"]]
y_train = players_training['played_hours']

X_test = players_testing[["age"]] 
y_test = players_testing['played_hours'] 

#make a pipeline for our model
players_pipe = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor()
)


marathon_cv = pd.DataFrame(
    cross_validate(
        players_pipe,
        X_train,
        y_train,
        scoring="neg_root_mean_squared_error",
        return_train_score=True
    )
)

#making grid parameters
param_grid = {'kneighborsregressor__n_neighbors': range(1, 30)}

#Doing cross validation
player_tuned = GridSearchCV(
    players_pipe,
    param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

##make a new dataframe from the results of our cross validation
player_results = pd.DataFrame(
    player_tuned.fit(X_train, y_train).cv_results_
)

#finding best number of K
player_min = player_tuned.best_params_
player_best_RMSPE = -player_tuned.best_score_

player_min

/opt/conda/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


NameError: name 'player_min' is not defined